## Traduzione da PYTHON a JULIA

*NOTE*
- Io uso julia a 32bit quindi di base mi prende Int32, costruisce le funzioni con Int32 all'occorrenza si possono modificare con Int64. (ad esempio il prof ha creato index2addr con int64 e non me la prende)

- I vertici vanno scritti in verticale, cioè con array senza virgole [[0,0] [0,1] [1,0] [1,1]] tipo matrici 

- le celle invece le scrivo come array con virgole quindi la cella è ad esempio [[1,2,3,4],[3,4,5,6]]

- i simplessi sono le corrispondenze di indici dei vertici dei triangoli esempio [[0,0] [1,0] [0,1] [1,1]], [[0,1,2],[1,2,3]]


### Funzione: *larSplit*

##### Python

In [ ]:
def larSplit(dom):
    def larSplit1(n):
        # assert n > 0 and isinstance(n,int)
        item = float(dom)/n
        ints = range(n+1)
        items = [item]*(n+1)
        vertices = [[int*item] for (int,item) in zip(ints,items)]
        return vertices
    return larSplit1

##### Julia

In [26]:
function larSplit(dom)
    function larSplit1(n)
        item = dom/n
        ints = range(0,n+1) 
        vertices=[ints*item;]
        return reshape(vertices,1,n+1)
    end
    return larSplit1
end
        

larSplit (generic function with 1 method)

*ESEMPIO*

In [27]:
larSplit(5)(4)

1×5 Array{Float64,2}:
 0.0  1.25  2.5  3.75  5.0

### Funzione: *larGrid*

##### Python

In [ ]:
def grid0(n):
    cells = AA(LIST)(range(n+1))
    return cells

def grid1(n):
    ints = range(n+1)
    cells = TRANS([ints[:-1],ints[1:]])
    return cells

def larGrid(n):
    def larGrid1(d):
        if d==0: return grid0(n)
        elif d==1: return grid1(n)
    return larGrid1

##### Julia

In [15]:
function grid0(n)
    return [[i] for i in range(1,n+1)] 
end

function grid1(n)
    return [[i,i+1] for i in range(1,n)]
end

function Cell(n)
    function Cell1(d)
        if d==0 
            return grid0(n)
        elseif d==1 
            return grid1(n) 
        end
    end
    return Cell1
end

function grid_0(n)
    return hcat([[i] for i in range(0,n+1)]...)
end

function grid_1(n)
    return hcat([[i,i+1] for i in range(0,n)]...)
end

function larGrid(n)
    function larGrid1(d)
        if d==0 
            return grid_0(n)
        elseif d==1 
            return grid_1(n) 
        end
    end
    return larGrid1
end

larGrid (generic function with 1 method)

*ESEMPIO*

In [16]:
grid_0(3)

4-element Array{Array{Int32,1},1}:
 [1]
 [2]
 [3]
 [4]

In [17]:
grid_1(3)

3-element Array{Array{Int32,1},1}:
 [1, 2]
 [2, 3]
 [3, 4]

In [19]:
larGrid(4)(0)

5-element Array{Array{Int32,1},1}:
 [1]
 [2]
 [3]
 [4]
 [5]

### Funzione: *larCuboidFacets*

##### Python

In [ ]:
def larCuboidsFacets((V,cells)):
   dim = len(V[0])
   n = int(2**(dim-1))
   facets = []
   for cell in cells:
      coords = [AR([V[v],v]) for v in cell] # decorate coords with vertex index
      doubleFacets = [sorted(coords,key=(lambda x: x[k])) for k in range(dim)] # copia la lista tot volte e la ordina rispetto agli elementi x[k] con k in range(dim)
      facets += AA(AA(LAST))(CAT([[pair[:n],pair[n:]] for pair in doubleFacets]))
   facets = AA(eval)(set(AA(str)(facets))) # remove duplicates
   return V,sorted(facets)


##### Julia

In [1]:
function larCuboidsFacets(V,cells)
    dim = size(V,1)
    n = 2^(dim-1)
    facets = []
    for cell in cells
        Vert=hcat([V[:,i] for i in cell]...)
        coords = vcat(Vert,reshape(cell,(1,size(cell)[1])))
        doubleFacets=hcat([coords[:,sortperm(coords[k,:])] for k in range(1,dim)]...)
        lastRow=doubleFacets[dim+1,:]
        facets0 = reshape(lastRow,(n,Int(size(lastRow)[1]/n)))
        append!(facets,collect([facets0[:,i] for i in range(1,size(facets0)[2])]))
    end
    facets = unique(facets)
    return V,sort(facets, by = x -> x[1])
end

larCuboidsFacets (generic function with 1 method)

*ESEMPIO*

In [3]:
larCuboidsFacets([[0, 0,0] [0, 1,0]  [1, 1,0]  [1, 0,0] [0, 0,1] [0, 1,1]  [1, 1,1]  [1, 0,1] ], [[1, 2, 3,4,5,6,7,8]])

([0 0 … 1 1; 0 1 … 1 0; 0 0 … 1 1], Array{Int32,1}[[1, 2, 5, 6], [1, 4, 5, 8], [1, 2, 3, 4], [2, 3, 6, 7], [3, 4, 7, 8], [5, 6, 7, 8]])

### Funzione: *larSimplicialStack*

##### Python

In [7]:
#NON nel mio modulo
def larSimplexFacets(simplices):
    out = []
    d = len(simplices[0])
    for simplex in simplices:
        out += AA(sorted)([simplex[0:k]+simplex[k+1:d] for k in range(d)])
    out = set(AA(tuple)(out))
    return  sorted(out)

def larSimplicialStack(simplices):
   dim = len(simplices[0])-1
   faceStack = [simplices]
   for k in range(dim):
      faces = larSimplexFacets(faceStack[-1])
      faceStack.append(faces)
   return REVERSE(faceStack)

LoadError: [91msyntax: extra token "larSimplexFacets" after end of expression[39m

##### Julia

In [5]:
function larSimplexFacets(simplices)#[[,],[,]]
    out=[]
    d=size(simplices[1],1)
    for simplex in simplices
        sim=vcat(simplex,simplex)
        for k = d-1
            append!(out,collect([sim[i:i+k-1] for i in range(1,d)]))
        end
    end
    facets0=collect([sort(out[i]) for i in range(1,length(out))])
    facets=unique(facets0)
    return sort(facets, by = x -> x[1])
end

function larSimplicialStack(simplices)
    dim=size(simplices[1],1)-1   
    faceStack = [simplices]
    for k in range(1,dim)
        faces = larSimplexFacets(faceStack[end])# errore nella chiamata ma p.funzione funziona
        append!(faceStack,[faces])
    end
    return flipdim(faceStack,1)
end       

larSimplicialStack (generic function with 1 method)

In [7]:
larSimplicialStack([[1,2,3,4]])

4-element Array{Array{Array{Int32,1},1},1}:
 Array{Int32,1}[[1], [2], [3], [4]]                            
 Array{Int32,1}[[1, 2], [1, 3], [1, 4], [2, 3], [2, 4], [3, 4]]
 Array{Int32,1}[[1, 2, 3], [1, 3, 4], [1, 2, 4], [2, 3, 4]]    
 Array{Int32,1}[[1, 2, 3, 4]]                                  

### Funzione: *larGridSkeleton*

##### Python

In [15]:
def index2addr(shape):
    n = len(shape)
    shape = shape[1:]+[1]
    weights = [PROD(shape[k:]) for k in range(n)]
    def index2addr0 (multindex):
        return INNERPROD([multindex, weights])
    return index2addr0

def larCellProd(cellLists):
    shapes = [len(item) for item in cellLists]
    indices = CART([range(shape) for shape in shapes])
    jointCells = [CART([cells[k] for k,cells in zip(index,cellLists)])
                  for index in indices]
    convert = index2addr([ shape+1 if (len(cellLists[k][0]) > 1) else shape
                             for k,shape in enumerate(shapes) ])
    return [AA(convert)(cell) for cell in jointCells]
    
def binaryRange(n):
    return [('{0:0'+str(n)+'b}').format(k) for k in range(2**n)]

def filterByOrder(n):
    terms = [AA(int)(list(term)) for term in binaryRange(n)]
    return [[term for term in terms if sum(term) == k] for k in range(n+1)]

def larGridSkeleton(shape):
    n = len(shape)
    def larGridSkeleton0(d):
        components = filterByOrder(n)[d]
        componentCellLists = [AA(APPLY)(zip( AA(larGrid)(shape),(component) ))
                              for component in components]
        return CAT([ larCellProd(cellLists)  for cellLists in componentCellLists ])
    return larGridSkeleton0

LoadError: [91msyntax: extra token "index2addr" after end of expression[39m

##### Julia

In [9]:
using IterTools

function index2addr(shape::Array{Int32,1})
    index2addr(hcat(shape...))
end

function index2addr(shape::Array{Int32,2})
    n = length(shape)
    theShape = append!(shape[2:end],1)
    weights = [prod(theShape[k:end]) for k in range(1,n)]
    function index2addr0(multiIndex)
        return dot(collect(multiIndex), weights) + 1
    end
    return index2addr0
end

function binaryRange(n)
    return [bin(k,n) for k in 0:2^n-1]
end

function filterByOrder(n)
    terms = [[parse(Int8,bit) for bit in convert(Array{Char,1},term)] for term in binaryRange(n)]
    return [[term for term in terms if sum(term) == k] for k in 0:n]
end

function cart(args)
   return sort(collect(product(args...)))
end

function larCellProd(cellLists)
    shapes = [length(item) for item in cellLists]
    subscripts = cart([collect(range(0,shape)) for shape in shapes])
    indices = hcat([collect(tuple) for tuple in subscripts]...)
    jointCells = Any[]
    for h in 1:size(indices,2)
        index = indices[:,h]
        cell = hcat(cart([cells[k+1] for (k,cells) in zip(index,cellLists)])...)
        append!(jointCells,[cell])
    end
    convertIt = index2addr([ (length(cellLists[k][1]) > 1)? shape+1 : shape for (k,shape) in enumerate(shapes) ])		
    [vcat(map(convertIt, jointCells[j])...) for j in 1:size(jointCells,1)]
    #[map(convertIt, jointCells[j]) for j in 1:size(jointCells,1)]
end

function larGridSkeleton(shape)
    n = length(shape)
    function larGridSkeleton0(d)
        components = filterByOrder(n)[d+1]
        mymap(arr) = [arr[:,k]  for k in 1:size(arr,2)]
        componentCellLists = [ [ mymap(f(x)) for (f,x) in zip( [larGrid(dim) for dim in shape],component ) ]
                              for component in components ]
        out = [ larCellProd(cellLists)  for cellLists in componentCellLists ]
        return vcat(out...)
    end
    return larGridSkeleton0
end                   

larGridSkeleton (generic function with 1 method)

### Funzione: *larModelProduct*

##### Python

In [ ]:
def larModelProduct(twoModels):
    (V, cells1), (W, cells2) = twoModels
    vertices = collections.OrderedDict(); k = 0
    for v in V:
        for w in W:
            id = tuple(v+w)
            if not vertices.has_key(id): #se quell'indice ha una chiave dice vero quindi not vero dice falso ed esce , se non ha una chiave allora entra e gliela da
                vertices[id] = k
                k += 1   
    cells = [ [vertices[tuple(V[v] + W[w])] for v in c1 for w in c2]
             for c1 in cells1 for c2 in cells2]  
    model = [list(v) for v in vertices.keys()], cells
    return model


##### Julia

In [10]:
using DataStructures


function larModelProduct(twoModels)
    modelOne, modelTwo = twoModels
    larModelProduct(modelOne, modelTwo)
end

function larModelProduct(modelOne, modelTwo)
    (V, cells1) = modelOne
    (W, cells2) = modelTwo

    k = 1
    vertices = OrderedDict()
    for v in V
        for w in W
            id = [v;w]
            if haskey(vertices, id) == false
                vertices[id] = k
                k = k + 1
            end
        end
    end

    cells = []
    for c1 in cells1
        for c2 in cells2
            cell = []
            for vc in c1
                for wc in c2 
                    push!(cell, vertices[[V[vc];W[wc]]] )
                end
            end
            push!(cells, cell)
        end
    end

    vertexmodel = []
    for v in keys(vertices)
        push!(vertexmodel, v)
    end

    return (vertexmodel, cells)
end


larModelProduct (generic function with 2 methods)

### Funzione: *larCuboids*

##### Python

In [ ]:
def larVertProd(vertLists):
    return AA(CAT)(CART(vertLists))

def larImageVerts(shape):
   def vertexDomain(n): 
      return [[k] for k in range(n)]
   vertLists = [vertexDomain(k+1) for k in shape]
   vertGrid = larVertProd(vertLists)
   return vertGrid

def larCuboids(shape, full=False):
   vertGrid = larImageVerts(shape)
   gridMap = larGridSkeleton(shape)
   if not full: 
      cells = gridMap(len(shape))
   else:
      skeletonIds = range(len(shape)+1)
      cells = [ gridMap(id) for id in skeletonIds ]
   return vertGrid, cells

##### Julia

In [11]:
using IterTools

function cart(args)
    return sort(collect(IterTools.product(args...)))
end

function larVertProd(vertLists)
    coords = [[x[1] for x in v] for v in cart(vertLists)]
   return sortcols(hcat(coords...))
end

function vertexDomain(n)
    return hcat([k for k in 0:n-1]...)
end

function larImageVerts(shape)
    vertLists = [vertexDomain(k+1) for k in shape]
    vertGrid = larVertProd(vertLists)
    return vertGrid
end

function larCuboids(shape, full=false)
    vertGrid = larImageVerts(shape)
    gridMap = larGridSkeleton(shape)
    if ! full
        cells = gridMap(length(shape))
    else
        skeletonIds = 0:length(shape)
        cells = [ gridMap(id) for id in skeletonIds ]
    end
    return vertGrid, cells
end


larCuboids (generic function with 2 methods)

ESEMPIO

In [3]:
shape = (1,1)
cubes = larCuboids(shape,true)
print(cubes)

LoadError: [91mUndefVarError: larCuboids not defined[39m

### Funzione: *gridSkeletons*

##### Python

In [ ]:
def gridSkeletons(shape):
   gridMap = larGridSkeleton(shape)
   skeletonIds = range(len(shape)+1)
   skeletons = [ gridMap(id) for id in skeletonIds ]
   return skeletons

##### Julia

In [12]:
function gridSkeletons(shape)
    gridMap = larGridSkeleton(shape)
    skeletonIds = range(0,length(shape)+1)
    skeletons = [gridMap(id) for id in skeletonIds]
    return skeletons
end

gridSkeletons (generic function with 1 method)